# Langgraph example with LiteLLM and SAP LLMs

## [How Langgraph works](https://docs.langchain.com/oss/python/langgraph/quickstart#use-the-functional-api)

## Installation

In [20]:
# %pip install langchain langgraph langchain-litellm
!cd ~/github/liteLLM && pip install -e .

Looking in indexes: https://pypi.org/simple, https://int.repositories.cloud.sap/artifactory/api/pypi/proxy-deploy-releases-hyperspace-pypi/simple
Obtaining file:///Users/d026073/github/litellm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for litellm (pyproject.toml) ... done
  Created wheel for litellm: filename=litellm-1.79.1-py3-none-any.whl size=9789 sha256=d3aa471033cd8d2c927f9ba703523da76e3aa304070be6d0dee27378fedf3422
  Stored in directory: /private/var/folders/84/d0z_jlpn7blc6rf673htv2j00000gn/T/pip-ephem-wheel-cache-gdgq16ui/wheels/17/c5/de/d7e8921fc6f30ef8c12d79d76a58b2835e04fdea787c103a1b
Successfully built litellm
  Attempting uninstall: litellm
    Found existing installation: litellm 1.79.3
    Uninstalling litellm-1.79.3:
      Successfully uninstalled litellm-1.79.3


## Credentials for SAP Gen AI Hub
Get the service key from your SAP BTP tenant with AI subscription.

Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
AICORE_AUTH_URL="https://* * * .authentication.sap.hana.ondemand.com/oauth/token",
AICORE_CLIENT_ID=" *** ",
AICORE_CLIENT_SECRET=" *** ",
AICORE_RESOURCE_GROUP=" *** ",
AICORE_BASE_URL="https://api.ai.***.cfapps.sap.hana.ondemand.com/
```

## Run the Langgraph with LiteLLM and SAP LLMs

In [1]:
from langchain.tools import tool
from langchain_litellm import ChatLiteLLM
from langgraph.graph import add_messages
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    ToolCall,
)
from langchain_core.messages import BaseMessage
from langgraph.func import entrypoint, task
from dotenv import load_dotenv

Load your credentials as environment variables that Litellm can use automatically.

In [2]:
load_dotenv()

True

Define the model with the SAP LLM.

In [3]:
model = ChatLiteLLM(model="sap/gpt-4o", temperature=0)

Define the agent tool

In [4]:
@tool
def get_weather(city: str):
    """
    Returns weather information for a given city.
    :param city:
    :return: weather information
    """
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

Augment the LLM with tools

In [5]:
tools = [get_weather]
tools_by_name = {tool.name: tool for tool in tools}
model_with_tools = model.bind_tools(tools)

Define model node

In [6]:
@task
def call_llm(messages: list[BaseMessage]):
    """LLM decides whether to call a tool or not"""
    return model_with_tools.invoke(
        [
            SystemMessage(
                content="You are a helpful weather assistant. "
                "When the user asks you about a specific city, "
                "use the 'get_weather' tool to find the information about the weather. "
                "Answer with a TV weather report in two sentences, including a small joke."
            )
        ]
        + messages
    )

Define tool node

In [7]:
@task
def call_tool(tool_call: ToolCall):
    """Performs the tool call"""
    tool = tools_by_name[tool_call["name"]]
    return tool.invoke(tool_call)

Define agent

In [8]:
@entrypoint()
def agent(messages: list[BaseMessage]):
    model_response = call_llm(messages).result()

    while True:
        if not model_response.tool_calls:
            break

        # Execute tools
        tool_result_futures = [
            call_tool(tool_call) for tool_call in model_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]
        messages = add_messages(messages, [model_response, *tool_results])
        model_response = call_llm(messages).result()

    messages = add_messages(messages, model_response)
    return messages

User can select a city.

In [13]:
city = 'London'

Run invoke

In [14]:
input_message = [HumanMessage(content=f"What's the weather in {city}?")]
for chunk in agent.stream(input_message, stream_mode="updates"):
    print(chunk)
    print("\n")

{'call_llm': AIMessage(content='', additional_kwargs={'tool_calls': [ChatCompletionMessageToolCall(function=Function(arguments='{"city":"New York"}', name='get_weather'), id='call_ojxxba3GQrzgOkJejSx2weJR', type='function')]}, response_metadata={'token_usage': Usage(completion_tokens=16, prompt_tokens=109, total_tokens=125, completion_tokens_details=None, prompt_tokens_details=None), 'model': 'sap/gpt-4o', 'finish_reason': 'tool_calls', 'model_name': 'sap/gpt-4o'}, id='run--50db4324-0e3d-4b26-8793-84cdebb7d53e-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'New York'}, 'id': 'call_ojxxba3GQrzgOkJejSx2weJR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 16, 'total_tokens': 125})}


{'call_tool': ToolMessage(content='The weather in New York is sunny with a temperature of 25°C.', name='get_weather', id='dfb58955-dcfb-4229-8144-d9a6dd7b7fc1', tool_call_id='call_ojxxba3GQrzgOkJejSx2weJR')}


{'call_llm': AIMessage(content="Good evening, New York! It'